In [1]:
import pandas as pd

In [107]:
import numpy as np

In [2]:
df1 = pd.read_csv('sampledata.csv')

D:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (2,3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df1.head()

,uid,device,song_id,song_type,song_name,singer,play_time,song_length,paid_flag,file_name,date,label
0,168551323.0,ar,1.68894e+07,0,一人饮酒醉唯美翻唱,沈雨萱,93,93,0.0,20170330_3_play.log,20170330.0,1
1,168542680.0,ar,0,1,心中的一道彩虹,何祺,284,285,0.0,20170330_3_play.log,20170330.0,0
2,168551542.0,ar,8.76228e+06,0,一笑倾城-(电视剧《微微一笑很倾城》片头曲),汪苏泷,120,232,0.0,20170330_3_play.log,20170330.0,1
3,168549948.0,ar,2.30825e+07,0,高尚,薛之谦,205,318,0.0,20170330_3_play.log,20170330.0,1
4,168551120.0,ar,2.28585e+07,0,像梦一样自由 (Live),汪峰,119,233,0.0,20170330_3_play.log,20170330.0,1


In [4]:
df1.isnull().sum()

uid                0
device             0
song_id         3183
song_type       7734
song_name       3074
singer         11708
play_time       6173
song_length    16924
paid_flag      14956
file_name      14956
date           14956
label              0
dtype: int64

In [6]:
df1 = df1.dropna(subset=['file_name'])
df1.isnull().sum()

uid               0
device            0
song_id        3183
song_type      6770
song_name      2101
singer         9449
play_time      3914
song_length    1969
paid_flag         0
file_name         0
date              0
label             0
dtype: int64

In [7]:
df1 = df1.dropna(subset=['play_time'])
df1.isnull().sum()

uid               0
device            0
song_id        1214
song_type      4801
song_name       137
singer         7484
play_time         0
song_length       0
paid_flag         0
file_name         0
date              0
label             0
dtype: int64

#### Song_id is used to calculate popularity score, Song_type is used to calculate number of genre, Song_name is used to calculate language. Song_type might be hard to determined sometimes so I just keep missing values of song type, therefore assign them 5 as the unknown type 

In [10]:
df1.shape

(6653680, 12)

In [12]:
df1 = df1.dropna(subset=['song_id'])
df1 = df1.dropna(subset=['song_name'])
df1.isnull().sum()

uid               0
device            0
song_id           0
song_type      3587
song_name         0
singer         7445
play_time         0
song_length       0
paid_flag         0
file_name         0
date              0
label             0
dtype: int64

In [13]:
df1.loc[df1.song_type.isnull(),'song_type'] = 5

In [14]:
df1.head()

,uid,device,song_id,song_type,song_name,singer,play_time,song_length,paid_flag,file_name,date,label
0,168551323.0,ar,1.68894e+07,0,一人饮酒醉唯美翻唱,沈雨萱,93,93,0.0,20170330_3_play.log,20170330.0,1
1,168542680.0,ar,0,1,心中的一道彩虹,何祺,284,285,0.0,20170330_3_play.log,20170330.0,0
2,168551542.0,ar,8.76228e+06,0,一笑倾城-(电视剧《微微一笑很倾城》片头曲),汪苏泷,120,232,0.0,20170330_3_play.log,20170330.0,1
3,168549948.0,ar,2.30825e+07,0,高尚,薛之谦,205,318,0.0,20170330_3_play.log,20170330.0,1
4,168551120.0,ar,2.28585e+07,0,像梦一样自由 (Live),汪峰,119,233,0.0,20170330_3_play.log,20170330.0,1


### Create User Matrix

In [15]:
unique_user_list = df1.uid.unique()

In [17]:
user_matrix = pd.DataFrame({'user': unique_user_list})

In [18]:
df1['for_count'] = 1

In [19]:
play_count = df1[['uid', 'for_count']].groupby(['uid']).count()

In [24]:
user_matrix = play_count.reset_index()

In [26]:
play_count = 0
user_matrix.shape

(25390, 3)

### Create preferred device

In [27]:
a = df1[['uid', 'device']].groupby(['uid', 'device']).size().reset_index().rename(columns={0:'count'})

In [39]:
def favorite(a, b):
#     define halftime device means that the favorite
    if a > 0.5 * b:
        return 'ar'
    else:
        return 'ip'
def favorite2(a, b):
#     define halftime device means that the favorite
    if a < 0.5 * b:
        return 'ar'
    else:
        return 'ip'

In [29]:
a = a.set_index(['uid']).join(user_matrix.set_index(['uid'])).reset_index()

In [30]:
a.head()

,uid,device,count,index,for_count
0,0.0,ar,193989,0,193989
1,533817.0,ar,82304,1,82304
2,751824.0,ar,7129,2,7129
3,924065.0,ar,29767,3,29767
4,1342999.0,ar,4004,4,4004


In [32]:
a[a['device'] == 'ar'].shape

(21327, 5)

In [33]:
a.shape

(25402, 5)

In [34]:
user_matrix.shape

(25390, 3)

In [36]:
a1 = a[a['device'] == 'ar']
a2 = a[a['device'] == 'ip']

In [37]:
a1['favorite_device'] = a1.apply(lambda row: favorite(row['count'], row['for_count']), axis = 1)

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [40]:
a2['favorite_device'] = a2.apply(lambda row: favorite2(row['count'], row['for_count']), axis = 1)

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [41]:
a1 = a1.drop(a1[a1.favorite_device == 'ip'].index)
a2 = a2.drop(a2[a2.favorite_device == 'ar'].index)

In [46]:
b = user_matrix

In [61]:
a = pd.concat([a1, a2])

In [62]:
user_matrix = a[['uid', 'for_count', 'favorite_device']]

In [63]:
user_matrix.shape

(25390, 3)

### Add churn label

In [64]:
example = df1.drop_duplicates(subset = 'uid')
example = example[['uid', 'label']]

In [66]:
example.shape

(25390, 2)

In [67]:
user_matrix = user_matrix.set_index(['uid']).join(example.set_index(['uid'])).reset_index()
# user_matrix = user_matrix.set_index(['user']).join(a.set_index(['uid'])).reset_index()

In [68]:
user_matrix.head()

,uid,for_count,favorite_device,label
0,0.0,193989,ar,0
1,533817.0,82304,ar,0
2,751824.0,7129,ar,0
3,924065.0,29767,ar,0
4,1342999.0,4004,ar,0


In [85]:
user_matrix['favorite_device'] = user_matrix.favorite_device.apply(lambda x: 1 if x == 'ar' else 0)

In [87]:
b = user_matrix

### song length and play time

In [70]:
df1['song_length'] = df1['song_length'].astype(str)

In [71]:
df1['song_length'] = df1['song_length'].apply(lambda symbol: 
                                              symbol.split('>')[0] 
                                              if '>' in symbol else symbol)

In [73]:
df1['song_length'] = df1['song_length'].astype(float)

In [78]:
df1['play_time'] = df1['play_time'].apply(lambda symbol: 
                                               symbol.split('>')[0]
                                               if '>' in symbol else symbol)

In [80]:
def is_float(input):
  try:
    num = float(input)
  except ValueError:
    return True
  return False

def is_int(input):
  try:
    num = int(input)
  except ValueError:
    return True
  return False

In [81]:
li = []
for item in df1['play_time']:
    if is_float(item) and is_int(item):
       li.append(item)

In [82]:
set(li)

{'\tTV\xe3\x82\xa2\xe3\x83\x8b\xe3\x83\xa1\xe3\x80\x8e\xe3\x83\xaf\xe3\x83\xb3\xe3\x83\x91\xe3\x83\xb3\xe3\x83\x9e\xe3\x83\xb3\xe3\x80\x8f\xe3\x82\xaa\xe3\x83\xaa\xe3\x82\xb8\xe3\x83\x8a\xe3\x83\xab\xe3\x82\xb5\xe3\x82\xa6\xe3\x83\xb3\xe3\x83\x89\xe3\x83\x88\xe3\x83\xa9\xe3\x83\x83\xe3\x82\xaf\xe3\x80\x8cOne Take Man\xe3\x80\x8d',
 'Ten Nararak',
 'ai\xe3\x81\xa8cheng',
 '\xe5\x88\x86\xe7\xba\xa7\xe5\x8a\xa0\xe5\xad\x97\xe5\xb9\x95\xe8\xbd\xbb\xe6\x9d\xbe\xe7\xbb\x83\xe5\x90\xac\xe5\x8a\x9b',
 '\xe5\x91\xa8\xe6\x9d\xb0\xe4\xbc\xa6',
 '\xe9\x99\x88\xe6\x85\xa7\xe5\xa8\xb4'}

In [83]:
df1 = df1.drop(df1[df1.play_time == '\tTV\xe3\x82\xa2\xe3\x83\x8b\xe3\x83\xa1\xe3\x80\x8e\xe3\x83\xaf\xe3\x83\xb3\xe3\x83\x91\xe3\x83\xb3\xe3\x83\x9e\xe3\x83\xb3\xe3\x80\x8f\xe3\x82\xaa\xe3\x83\xaa\xe3\x82\xb8\xe3\x83\x8a\xe3\x83\xab\xe3\x82\xb5\xe3\x82\xa6\xe3\x83\xb3\xe3\x83\x89\xe3\x83\x88\xe3\x83\xa9\xe3\x83\x83\xe3\x82\xaf\xe3\x80\x8cOne Take Man\xe3\x80\x8d'].index)
df1 = df1.drop(df1[df1.play_time == 'Ten Nararak'].index)
df1 = df1.drop(df1[df1.play_time == 'ai\xe3\x81\xa8cheng'].index)
df1 = df1.drop(df1[df1.play_time == '\xe5\x88\x86\xe7\xba\xa7\xe5\x8a\xa0\xe5\xad\x97\xe5\xb9\x95\xe8\xbd\xbb\xe6\x9d\xbe\xe7\xbb\x83\xe5\x90\xac\xe5\x8a\x9b'].index)
df1 = df1.drop(df1[df1.play_time == '\xe5\x91\xa8\xe6\x9d\xb0\xe4\xbc\xa6'].index)
df1 = df1.drop(df1[df1.play_time == '\xe9\x99\x88\xe6\x85\xa7\xe5\xa8\xb4'].index)

In [84]:
df1['play_time'] = df1['play_time'].astype(float)

In [88]:
c = df1[['uid', 'play_time']]
c = c.groupby(['uid'])['play_time'].sum().reset_index()

In [90]:
user_matrix = user_matrix.set_index(['uid']).join(c.set_index(['uid'])).reset_index()

In [92]:
user_matrix.head()

,uid,for_count,favorite_device,label,play_time
0,0.0,193989,1,0,1.052149e+10
1,533817.0,82304,1,0,9.345974e+08
2,751824.0,7129,1,0,2.308999e+09
3,924065.0,29767,1,0,3.173783e+08
4,1342999.0,4004,1,0,3.983850e+05


In [93]:
bb = user_matrix

### ratio

In [94]:
df1['completion_ratio'] = df1['play_time']/ df1['song_length']

In [95]:
df1.head()

,uid,device,song_id,song_type,song_name,singer,play_time,song_length,paid_flag,file_name,date,label,for_count,completion_ratio
0,168551323.0,ar,1.68894e+07,0,一人饮酒醉唯美翻唱,沈雨萱,93.0,93.0,0.0,20170330_3_play.log,20170330.0,1,1,1.000000
1,168542680.0,ar,0,1,心中的一道彩虹,何祺,284.0,285.0,0.0,20170330_3_play.log,20170330.0,0,1,0.996491
2,168551542.0,ar,8.76228e+06,0,一笑倾城-(电视剧《微微一笑很倾城》片头曲),汪苏泷,120.0,232.0,0.0,20170330_3_play.log,20170330.0,1,1,0.517241
3,168549948.0,ar,2.30825e+07,0,高尚,薛之谦,205.0,318.0,0.0,20170330_3_play.log,20170330.0,1,1,0.644654
4,168551120.0,ar,2.28585e+07,0,像梦一样自由 (Live),汪峰,119.0,233.0,0.0,20170330_3_play.log,20170330.0,1,1,0.510730


In [98]:
df1 = df1.dropna(subset=['completion_ratio'])

In [116]:
c = df1[['uid', 'completion_ratio']]
# c = c.groupby(['uid']).mean().reset_index()

In [117]:
c = c.replace([np.inf, -np.inf], np.nan)
# c[c['completion_ratio'] > 4]

In [118]:
c = c.dropna(subset=['completion_ratio'])

In [119]:
c = c.groupby(['uid']).mean().reset_index()

In [121]:
# c[c['completion_ratio'] < 0]

In [123]:
user_matrix = user_matrix.set_index(['uid']).join(c.set_index(['uid'])).reset_index()

In [124]:
user_matrix.head()

,uid,for_count,favorite_device,label,play_time,completion_ratio
0,0.0,193989,1,0,1.052149e+10,129.218331
1,533817.0,82304,1,0,9.345974e+08,36.393828
2,751824.0,7129,1,0,2.308999e+09,-321123.118894
3,924065.0,29767,1,0,3.173783e+08,34.594272
4,1342999.0,4004,1,0,3.983850e+05,0.533570


### Window

In [125]:
window = df1[['uid','date','song_id']].groupby(['uid','date'],as_index=False).count().rename(columns={'song_id': 'count'})

In [126]:
window1 =  window.query('20170421.0 <= date <= 20170421.0')
window2 =  window.query('date in (20170421.0,20170420.0,20170419.0)')
window3 =  window.query('20170415.0 <= date <= 20170421.0')
window4 =  window.query('20170408.0 <= date <= 20170421.0')
window5 =  window.query('20170330.0 <= date <= 20170421.0')

In [127]:
window1 = window1.groupby(["uid"], as_index=False)['count'].sum()
window2 = window2.groupby(["uid"], as_index=False)['count'].sum()
window3 = window3.groupby(["uid"], as_index=False)['count'].sum()
window4 = window4.groupby(["uid"], as_index=False)['count'].sum()
window5 = window5.groupby(["uid"], as_index=False)['count'].sum()

In [128]:
b = user_matrix

In [139]:
user_matrix = b

In [140]:
user_matrix = user_matrix.set_index(['uid']).join(window1.set_index(['uid'])).reset_index()

In [141]:
user_matrix = user_matrix.rename(columns={'count': 'win1'})

In [142]:
user_matrix = user_matrix.set_index(['uid']).join(window2.set_index(['uid'])).reset_index()
user_matrix = user_matrix.rename(columns={'count': 'win2'})

In [143]:
user_matrix = user_matrix.set_index(['uid']).join(window3.set_index(['uid'])).reset_index()
user_matrix = user_matrix.rename(columns={'count': 'win3'})

In [144]:
user_matrix = user_matrix.set_index(['uid']).join(window4.set_index(['uid'])).reset_index()
user_matrix = user_matrix.rename(columns={'count': 'win4'})

In [145]:
user_matrix = user_matrix.set_index(['uid']).join(window5.set_index(['uid'])).reset_index()
user_matrix = user_matrix.rename(columns={'count': 'win5'})

In [146]:
user_matrix.head()

,uid,for_count,favorite_device,label,play_time,completion_ratio,win1,win2,win3,win4,win5
0,0.0,193989,1,0,1.052149e+10,129.218331,16142.0,29143.0,74499.0,141200.0,182377.0
1,533817.0,82304,1,0,9.345974e+08,36.393828,2940.0,10078.0,24079.0,46313.0,75329.0
2,751824.0,7129,1,0,2.308999e+09,-321123.118894,NaN,NaN,1644.0,6880.0,6880.0
3,924065.0,29767,1,0,3.173783e+08,34.594272,516.0,764.0,2186.0,24489.0,29103.0
4,1342999.0,4004,1,0,3.983850e+05,0.533570,552.0,745.0,1316.0,2770.0,4000.0


### paid type

In [147]:
paid = df1[['uid','paid_flag','song_id']].groupby(['uid','paid_flag'],as_index=False).count().rename(columns={'song_id': 'count'})

In [149]:
paid[paid['paid_flag'] == 1]

,uid,paid_flag,count


### language

In [150]:
def chinese(a):
    for ch in a:
        if u'\u4e00' <= ch <= u'\u9fff':
            return 1
    return 0

In [154]:
b = user_matrix

In [151]:
song_name = df1['song_name']

In [152]:
song_name = list(song_name)

In [153]:
song_name_list = []
cc = 0
for i in song_name:
    try:
        song_name_list.append(chinese(i))
    except:
        song_name_list.append(1)

In [155]:
user_matrix['song language'] = pd.Series(song_name_list)

In [156]:
user_matrix.head()

,uid,for_count,favorite_device,label,play_time,completion_ratio,win1,win2,win3,win4,win5,song language
0,0.0,193989,1,0,1.052149e+10,129.218331,16142.0,29143.0,74499.0,141200.0,182377.0,1
1,533817.0,82304,1,0,9.345974e+08,36.393828,2940.0,10078.0,24079.0,46313.0,75329.0,1
2,751824.0,7129,1,0,2.308999e+09,-321123.118894,NaN,NaN,1644.0,6880.0,6880.0,1
3,924065.0,29767,1,0,3.173783e+08,34.594272,516.0,764.0,2186.0,24489.0,29103.0,1
4,1342999.0,4004,1,0,3.983850e+05,0.533570,552.0,745.0,1316.0,2770.0,4000.0,1


### Song type

In [161]:
b = user_matrix

In [171]:
genre = df1[['uid', 'song_type']].groupby(['uid', 'song_type']).size()

In [172]:
genre = genre.reset_index()
genre.colnames = ['uid', 'song_type', 'count']

In [173]:
genre.columns = ['uid', 'song_type', 'count'] 

In [174]:
favorite_genre = genre.ix[genre.groupby(['uid'], sort=False)['count'].idxmax()]

D:\Anaconda\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


In [175]:
favorite_genre = favorite_genre[['uid', 'song_type']]

In [176]:
user_matrix = user_matrix.set_index(['uid']).join(favorite_genre.set_index(['uid'])).reset_index()

In [177]:
user_matrix.head()

,uid,for_count,favorite_device,label,play_time,completion_ratio,win1,win2,win3,win4,win5,song language,song_type
0,0.0,193989,1,0,1.052149e+10,129.218331,16142.0,29143.0,74499.0,141200.0,182377.0,1,0
1,533817.0,82304,1,0,9.345974e+08,36.393828,2940.0,10078.0,24079.0,46313.0,75329.0,1,0
2,751824.0,7129,1,0,2.308999e+09,-321123.118894,NaN,NaN,1644.0,6880.0,6880.0,1,1
3,924065.0,29767,1,0,3.173783e+08,34.594272,516.0,764.0,2186.0,24489.0,29103.0,1,0
4,1342999.0,4004,1,0,3.983850e+05,0.533570,552.0,745.0,1316.0,2770.0,4000.0,1,0


In [178]:
user_matrix.to_csv('feature.csv',encoding="utf-8",index = False)